# Card Classifier Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path

In [5]:
class SmallCardNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)   # adjust 8×8 to your resized size
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [9]:
data_root = Path().resolve().parents[1] / 'data' / 'card_classifier' / 'pbs'
data_root

PosixPath('/home/eaglehawkinator/dev/king-tensor/data/card_classifier/pbs')

In [13]:

# basic transforms – you can add more augmentation later
train_tfms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(),      # optional
    transforms.ColorJitter(0.1, 0.1, 0.1),  # optional
    transforms.ToTensor(),                  # [0,1]
])

val_tfms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder(root=data_root, transform=train_tfms)
# val_ds   = datasets.ImageFolder(root=data_root / "val",   transform=val_tfms)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)
# val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False, num_workers=4)

num_classes = len(train_ds.classes)
print("Classes:", train_ds.classes)

Classes: ['bandit', 'battle_ram', 'electro_wizard', 'minions', 'pekka', 'poison', 'royal_ghost', 'zap']


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# num_classes from your ImageFolder
num_classes = len(train_ds.classes)

model = SmallCardNet(num_classes).to(device)

criterion = nn.CrossEntropyLoss()               # for multi-class classification
optimizer = torch.optim.Adam(model.parameters(), 
                              lr=1e-3, 
                              weight_decay=1e-4)

num_epochs = 150

Device: cpu


In [18]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # forward
        outputs = model(images)          # [batch, num_classes]
        loss = criterion(outputs, labels)

        # backward + update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # stats
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    print(f"Epoch {epoch+1}/{num_epochs} "
          f"- loss: {epoch_loss:.4f} - acc: {epoch_acc:.4f}")


Epoch 1/150 - loss: 2.0967 - acc: 0.1034
Epoch 2/150 - loss: 2.0543 - acc: 0.1379
Epoch 3/150 - loss: 2.0076 - acc: 0.2414
Epoch 4/150 - loss: 1.9675 - acc: 0.2414
Epoch 5/150 - loss: 1.9572 - acc: 0.2414
Epoch 6/150 - loss: 1.9589 - acc: 0.2414
Epoch 7/150 - loss: 1.9384 - acc: 0.2414
Epoch 8/150 - loss: 1.9227 - acc: 0.2414
Epoch 9/150 - loss: 1.9073 - acc: 0.2414
Epoch 10/150 - loss: 1.8999 - acc: 0.2414
Epoch 11/150 - loss: 1.8845 - acc: 0.2414
Epoch 12/150 - loss: 1.8613 - acc: 0.2414
Epoch 13/150 - loss: 1.8430 - acc: 0.2414
Epoch 14/150 - loss: 1.7900 - acc: 0.4138
Epoch 15/150 - loss: 1.7407 - acc: 0.5172
Epoch 16/150 - loss: 1.7041 - acc: 0.4828
Epoch 17/150 - loss: 1.6524 - acc: 0.4138
Epoch 18/150 - loss: 1.5379 - acc: 0.4828
Epoch 19/150 - loss: 1.4604 - acc: 0.5862
Epoch 20/150 - loss: 1.3265 - acc: 0.6207
Epoch 21/150 - loss: 1.2429 - acc: 0.6897
Epoch 22/150 - loss: 1.1058 - acc: 0.7586
Epoch 23/150 - loss: 0.9961 - acc: 0.7586
Epoch 24/150 - loss: 0.8756 - acc: 0.7931
E

In [19]:
def predict_card(model, img_path, classes, tfm):
    img = Image.open(img_path).convert("RGB")
    x = tfm(img).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        logits = model(x.to(next(model.parameters()).device))
    idx = logits.argmax(1).item()
    return classes[idx]

In [20]:
model_name = 'pbs_naive.pth'
save_path = Path().resolve().parents[1] / 'king_tensor' / 'models' / model_name
torch.save(model.state_dict(), save_path)